# example code from Qiita - Python+PuLPによるタダで仕事に使える数理最適化

https://qiita.com/samuelladoco/items/703bf78ea66e8369c455

## 問題(1): 線形最適化問題（線形計画問題）
最大化 x+y

制約
2x+y≤2
x+2y≤2
x≥0
y≥0

In [24]:
import pulp
import sys

In [61]:
pulp.pulpTestAll()

	 Testing zero subtraction
	 Testing inconsistant lp solution
	 Testing continuous LP solution
	 Testing maximize continuous LP solution
	 Testing unbounded continuous LP solution
	 Testing Long Names
	 Testing repeated Names
	 Testing zero constraint
	 Testing zero objective
	 Testing LpVariable (not LpAffineExpression) objective
	 Testing Long lines in LP
	 Testing LpAffineExpression divide
	 Testing MIP solution
	 Testing MIP solution with floats in objective
	 Testing MIP relaxation
	 Testing feasibility problem (no objective)
	 Testing an infeasible problem
	 Testing an integer infeasible problem
	 Testing column based modelling
	 Testing dual variables and slacks reporting
	 Testing fractional constraints
	 Testing elastic constraints (no change)
	 Testing elastic constraints (freebound)
	 Testing elastic constraints (penalty unchanged)
	 Testing elastic constraints (penalty unbounded)
* Solver <class 'pulp.solvers.PULP_CBC_CMD'> passed.
Solver <class 'pulp.solvers.CPLEX_DLL'> un

In [63]:
pulp.LpStatus

{0: 'Not Solved',
 1: 'Optimal',
 -1: 'Infeasible',
 -2: 'Unbounded',
 -3: 'Undefined'}

In [60]:
pulp.LpContinuous

'Continuous'

In [25]:
problem = pulp.LpProblem('problem 1', pulp.LpMaximize)
x = pulp.LpVariable('x', 0, 999, pulp.LpContinuous)
y = pulp.LpVariable('y', 0, sys.maxsize, pulp.LpContinuous)
# sys.maxsize == 9223372036854775807
problem += (x+y, 'Objective')
problem += (2*x + y <= 2, "Constraint_1")
problem += (x + 2*y <= 2, "Constraint_2")
print(problem)


problem 1:
MAXIMIZE
1*x + 1*y + 0
SUBJECT TO
Constraint_1: 2 x + y <= 2

Constraint_2: x + 2 y <= 2

VARIABLES
x <= 999 Continuous
y <= 9.22337203685e+18 Continuous



In [26]:
%%time
result_status = problem.solve()

CPU times: user 4.66 ms, sys: 12.1 ms, total: 16.8 ms
Wall time: 25.5 ms


In [27]:
result_status

1

In [28]:
pulp.LpStatus[result_status]

'Optimal'

In [29]:
pulp.value(problem.objective)

1.33333334

In [30]:
pulp.value(x), pulp.value(y)

(0.66666667, 0.66666667)

## 問題(2): 整数最適化問題（整数計画問題）

In [31]:
import pulp

I = ['A', "B", "C"]
J = ['work1', "work2", "work3"]

costIJ_WorkersWork = [
    [1,2,3],
    [4,6,8],
    [10,13,16]
]

print("workers I == {0}".format(I))
print("works J == {0}".format(J))
print("cost costIJ_WorkersWork == {0}".format(costIJ_WorkersWork))
costDict = {}
for i in I:
    for j in J:
        costDict[i,j] = costIJ_WorkersWork[I.index(i)][J.index(j)]
print("costDict == {0}".format(costDict))

workers I == ['A', 'B', 'C']
works J == ['work1', 'work2', 'work3']
cost costIJ_WorkersWork == [[1, 2, 3], [4, 6, 8], [10, 13, 16]]
costDict == {('A', 'work1'): 1, ('A', 'work2'): 2, ('A', 'work3'): 3, ('B', 'work1'): 4, ('B', 'work2'): 6, ('B', 'work3'): 8, ('C', 'work1'): 10, ('C', 'work2'): 13, ('C', 'work3'): 16}


In [32]:
costDict

{('A', 'work1'): 1,
 ('A', 'work2'): 2,
 ('A', 'work3'): 3,
 ('B', 'work1'): 4,
 ('B', 'work2'): 6,
 ('B', 'work3'): 8,
 ('C', 'work1'): 10,
 ('C', 'work2'): 13,
 ('C', 'work3'): 16}

In [33]:
problem = pulp.LpProblem("problem2", pulp.LpMinimize)
x = {}
for i in I:
    for j in J:
        x[i,j] = pulp.LpVariable("x({:},{:})".format(i,j), 
                                 0, 1, 
                                 pulp.LpInteger)
x   

{('A', 'work1'): x(A,work1),
 ('A', 'work2'): x(A,work2),
 ('A', 'work3'): x(A,work3),
 ('B', 'work1'): x(B,work1),
 ('B', 'work2'): x(B,work2),
 ('B', 'work3'): x(B,work3),
 ('C', 'work1'): x(C,work1),
 ('C', 'work2'): x(C,work2),
 ('C', 'work3'): x(C,work3)}

In [34]:
problem += pulp.lpSum(costDict[i,j]*x[i,j] for i in I for j in J), "TotalCost"

for i in I:
    problem += sum(x[i,j] for j in J)<=1, "constraint_leq_{:}".format(i)
for j in J:
    problem += sum(x[i,j] for i in I)==1, "constraint_eq_{:}".format(j)
print(problem)


problem2:
MINIMIZE
1*x(A,work1) + 2*x(A,work2) + 3*x(A,work3) + 4*x(B,work1) + 6*x(B,work2) + 8*x(B,work3) + 10*x(C,work1) + 13*x(C,work2) + 16*x(C,work3) + 0
SUBJECT TO
constraint_leq_A: x(A,work1) + x(A,work2) + x(A,work3) <= 1

constraint_leq_B: x(B,work1) + x(B,work2) + x(B,work3) <= 1

constraint_leq_C: x(C,work1) + x(C,work2) + x(C,work3) <= 1

constraint_eq_work1: x(A,work1) + x(B,work1) + x(C,work1) = 1

constraint_eq_work2: x(A,work2) + x(B,work2) + x(C,work2) = 1

constraint_eq_work3: x(A,work3) + x(B,work3) + x(C,work3) = 1

VARIABLES
0 <= x(A,work1) <= 1 Integer
0 <= x(A,work2) <= 1 Integer
0 <= x(A,work3) <= 1 Integer
0 <= x(B,work1) <= 1 Integer
0 <= x(B,work2) <= 1 Integer
0 <= x(B,work3) <= 1 Integer
0 <= x(C,work1) <= 1 Integer
0 <= x(C,work2) <= 1 Integer
0 <= x(C,work3) <= 1 Integer



In [36]:
%%time
solver = pulp.solvers.PULP_CBC_CMD()
result_status = problem.solve(solver)

CPU times: user 9.97 ms, sys: 4.78 ms, total: 14.7 ms
Wall time: 35.2 ms


In [37]:
print(pulp.LpStatus[result_status], pulp.value(problem.objective))
for i in I:
    for j in J:
        print(x[i,j].name, x[i,j].value())

Optimal 19.0
x(A,work1) 0.0
x(A,work2) 0.0
x(A,work3) 1.0
x(B,work1) 0.0
x(B,work2) 1.0
x(B,work3) 0.0
x(C,work1) 1.0
x(C,work2) 0.0
x(C,work3) 0.0


In [40]:
pulp.value(x["A", "work1"]), x["A", "work1"].value()


(0.0, 0.0)

# Qiita example - 最適化におけるPython
https://qiita.com/SaitoTsutomu/items/070ca9cb37c6b2b492f0


In [45]:
import pulp

In [43]:
m = pulp.LpProblem(sense=pulp.LpMaximize)
x = pulp.LpVariable('x', lowBound=0)
y = pulp.LpVariable('y', lowBound=0)

m += 100 * x + 100 * y

m += x + 2 * y <= 16
m += 3 * x + y <= 18
m.solve()
print(pulp.value(x), pulp.value(y))


4.0 6.0


## 輸送最適化問題

- 倉庫群から工場群への輸送量を決めたい → 変数
- 輸送コストを最小化したい → 目的関数
- 各倉庫からの搬出は、供給可能量以下 → 制約
- 各工場への搬入は、需要量以上 → 制約


In [46]:
import numpy as np
import pandas as pd
from itertools import product
from pulp import *

In [47]:
# next code
np.random.seed(1)
nw, nf = 3, 4
pr = list(product(range(nw), range(nf)))
print(pr)


[(0, 0), (0, 1), (0, 2), (0, 3), (1, 0), (1, 1), (1, 2), (1, 3), (2, 0), (2, 1), (2, 2), (2, 3)]


In [49]:
supply = np.random.randint(30, 50, nw)
demand = np.random.randint(20, 40, nf)
cost = np.random.randint(10, 20, (nw, nf))
print(supply, demand, cost)

[34 39 47] [20 33 29 29] [[17 16 19 11]
 [10 11 18 18]
 [13 19 18 17]]


### not use pandas case

In [50]:


m1 = pulp.LpProblem()
v1 = {(i, j): pulp.LpVariable('v%d_%d' % (i, j), lowBound=0) for i, j in pr}

m1 += pulp.lpSum(cost[i][j] * v1[i, j] for i, j in pr)

for i in range(nw):
    m1 += pulp.lpSum(v1[i, j] for j in range(nf)) <= supply[i]
for j in range(nf):
    m1 += pulp.lpSum(v1[i, j] for i in range(nw)) >= demand[j]



In [51]:
%%time
m1.solve()

CPU times: user 393 µs, sys: 16.1 ms, total: 16.5 ms
Wall time: 24.8 ms


1

In [52]:
print({k: pulp.value(x) for k, x in v1.items() if pulp.value(x) > 0})

{(0, 3): 29.0, (1, 0): 6.0, (1, 1): 33.0, (2, 0): 14.0, (2, 2): 29.0}


### use pandas case

In [56]:
a = pd.DataFrame([(i, j) for i, j in pr], 
                 columns=['warehouse', "factory"])
a['cost'] = cost.flatten()
print(a)


    warehouse  factory  cost
0           0        0    17
1           0        1    16
2           0        2    19
3           0        3    11
4           1        0    10
5           1        1    11
6           1        2    18
7           1        3    18
8           2        0    13
9           2        1    19
10          2        2    18
11          2        3    17


In [57]:
m2 = pulp.LpProblem()
a['Var'] = [pulp.LpVariable('v%d' % i, lowBound=0) for i in a.index]

m2 += pulp.lpDot(a.cost, a.Var)

for k, v in a.groupby('warehouse'):
    m2 += pulp.lpSum(v.Var) <= supply[k]
for k, v in a.groupby('factory'):
    m2 += pulp.lpSum(v.Var) >= demand[k]



In [58]:
%%time
m2.solve()


CPU times: user 5.43 ms, sys: 8.2 ms, total: 13.6 ms
Wall time: 26.2 ms


1

In [59]:
a['Val'] = a.Var.apply(pulp.value)
a[a.Val > 0]
print(a)

    warehouse  factory  cost  Var   Val
0           0        0    17   v0   0.0
1           0        1    16   v1   0.0
2           0        2    19   v2   0.0
3           0        3    11   v3  29.0
4           1        0    10   v4   6.0
5           1        1    11   v5  33.0
6           1        2    18   v6   0.0
7           1        3    18   v7   0.0
8           2        0    13   v8  14.0
9           2        1    19   v9   0.0
10          2        2    18  v10  29.0
11          2        3    17  v11   0.0
